<a href="https://colab.research.google.com/github/myy04/Real-Fake-Image-Classifier/blob/main/resnet18_classifier_YonseiTrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import AutoModel, AutoProcessor
import os
import torchvision
import torchvision.transforms as transforms
from torch import Tensor

import warnings
warnings.filterwarnings("ignore")

In [41]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ciplab/real-and-fake-face-detection")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/ciplab/real-and-fake-face-detection/versions/1


In [42]:
class ImageDataset(Dataset):
    def __init__(self, path):

        real_images_directory = os.path.join(path, 'training_real')
        fake_images_directory = os.path.join(path, 'training_fake')

        real_images = []
        fake_images = []

        for file in os.listdir(real_images_directory):
            real_images.append(os.path.join(real_images_directory, file))

        for file in os.listdir(fake_images_directory):
            fake_images.append(os.path.join(fake_images_directory, file))

        self.images = []
        for i in range(min(len(real_images), len(fake_images))):
            self.images.append((real_images[i], torch.tensor([1, 0], dtype = torch.float32)))
            self.images.append((fake_images[i], torch.tensor([0, 1], dtype = torch.float32)))

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to a standard size for ResNet
            transforms.ToTensor(),          # Convert PIL Image to PyTorch Tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx][0]
        image = Image.open(image_path)

        image = self.transform(image)

        return image, self.images[idx][1]


BATCHSIZE = 512
NEPOCH = 20

from torch.utils.data import random_split

train_dataset = ImageDataset(os.path.join(path, 'real_and_fake_face'))

# Split the data
train_size = int(0.8 * len(train_dataset))  # 80% for training
eval_size = len(train_dataset) - train_size  # Remaining 20% for evaluation
train_dataset, eval_dataset = random_split(train_dataset, [train_size, eval_size])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size = BATCHSIZE, shuffle = True)
eval_dataloader = DataLoader(eval_dataset, batch_size = BATCHSIZE, shuffle = True)


In [43]:
resnet_model = torchvision.models.resnet18(pretrained = True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_model = resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr = 1e-3)

In [44]:
def train_step(input: Tensor, label: Tensor):
  resnet_model.train()

  input = input.to(device)
  label = label.to(device)

  output = resnet_model(input)
  loss = criterion(output, label)

  optimizer.zero_grad()
  loss.backward()

  torch.nn.utils.clip_grad_norm_(resnet_model.parameters(), max_norm=1.0)
  optimizer.step()

  return loss, output

In [45]:
@torch.no_grad()
def test_step(input: Tensor, label: Tensor):

    resnet_model.eval()

    input = input.to(device)
    label = label.to(device)

    output = resnet_model(input)

    loss = criterion(output, label)

    return loss, output

In [46]:
def train_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = train_step(input, label)

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        # The label tensor has shape (batch_size, 2) - we need to compare predicted to the index with the maximum value in the label tensor
        correct += (predicted == torch.argmax(label, dim=1).to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")

    print(f"Train Accuracy: {correct / samples * 100.00}%")

In [47]:
def test_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = test_step(input, label)

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        correct += (predicted == torch.argmax(label, dim=1).to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")

    print(f"Test Accuracy: {correct / samples * 100.00}%")

In [48]:
for epoch in range(NEPOCH):
    train_epoch(train_dataloader)

    test_epoch(eval_dataloader)

torch.save(resnet_model.state_dict(), 'resnet_model_parameters.pth')

Loss [0/3]: 0.7435621023178101
Train Accuracy: 50.651041666666664%
Loss [0/1]: 0.7256205081939697
Test Accuracy: 52.604166666666664%
Loss [0/3]: 0.7376407384872437
Train Accuracy: 50.520833333333336%
Loss [0/1]: 0.7083730101585388
Test Accuracy: 54.947916666666664%
Loss [0/3]: 0.7158883213996887
Train Accuracy: 50.520833333333336%
Loss [0/1]: 0.6953555941581726
Test Accuracy: 54.427083333333336%
Loss [0/3]: 0.7277482748031616
Train Accuracy: 50.651041666666664%
Loss [0/1]: 0.688300609588623
Test Accuracy: 55.46875%
Loss [0/3]: 0.7332004308700562
Train Accuracy: 51.302083333333336%
Loss [0/1]: 0.6855652332305908
Test Accuracy: 55.989583333333336%
Loss [0/3]: 0.7099263668060303
Train Accuracy: 51.236979166666664%
Loss [0/1]: 0.6850180625915527
Test Accuracy: 55.989583333333336%
Loss [0/3]: 0.7206705808639526
Train Accuracy: 52.1484375%
Loss [0/1]: 0.6851096153259277
Test Accuracy: 55.46875%
Loss [0/3]: 0.717328667640686
Train Accuracy: 52.213541666666664%
Loss [0/1]: 0.6850537061691284
T